In [1]:
import os
import torch 
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from packages.vocab import Vocab
from packages.batch import Batch
from models.languagemodel import RNNLM

In [2]:
# Hyper Parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 10
num_samples = 1000   # number of words to be sampled
batch_size = 100
seq_length = 50
learning_rate = 0.002

In [3]:
train_path = '/home/irteam/users/data/150kJavaScript/data_preprocessed/'
with open(os.path.join(train_path,'list_of_files.txt')) as f:
    file_list = f.read().split('\n')

In [4]:
batch = Batch(file_dir=train_path,file_list=file_list,batch_size=batch_size,
              in_seq=50,out_seq=50)

In [5]:
import pickle
with open('vocabs/id_counter.pckl','rb') as f:
    id_counter = pickle.load(f)
with open('vocabs/tok_counter.pckl','rb') as f:
    tok_counter = pickle.load(f)
from slimit.lexer import Lexer
lexer = Lexer()

In [6]:
word_list = [x for (x,_) in tok_counter.most_common(100)] + ['STRING'] + [x for (x,_) in id_counter.most_common(10000)]

In [7]:
vocab = Vocab(10000)

In [8]:
vocab.add_to_vocab(word_list)
# vocab.w2i = np.load('vocabs/word2idx_no_ids.txt')
# vocab.i2w = np.load('vocabs/idx2word_no_ids.txt')
# vocab.max_size = len(vocab.w2i)

Vocabulary max size reached!


In [9]:
model = torch.load('models/rnn_lm_types_2_epoch_10.pckl')
# model = RNNLM(vocab.max_size, embed_size=embed_size, hidden_size=hidden_size,
#              num_layers = num_layers)
model.cuda()

RNNLM (
  (embed): Embedding(10000, 128)
  (lstm): LSTM(128, 1024, batch_first=True)
  (linear): Linear (1024 -> 10000)
)

In [10]:
# Truncated Backpropagation 
def detach(states):
    return [state.detach() for state in states] 

In [16]:
total = 0
correct = 0
# Training
batch.next_epoch(batch_size)
batch.initialize_states(num_layers, hidden_size)
total_steps=100
step=0
while(batch.epoch_end==0):
    step+=1
    # update the minibatch inputs / outputs
    batch.get_minibatch(0)
    inputs_np = np.array([vocab.word_list_to_idx_list(line) for
                         line in batch.batch_in],dtype=int)
    targets_np = np.array([vocab.word_list_to_idx_list(line) for
                         line in batch.batch_out],dtype=int)
    inputs = Variable(torch.LongTensor(inputs_np)).cuda()
    targets = Variable(torch.LongTensor(targets_np)).cuda()

    outputs, states = model(inputs,batch.states)
    out = outputs.view(targets.size(0),targets.size(1),-1).max(2)[1]
    batch.states = detach(states)
    
    total_0 = (targets==0).data.cpu().numpy().sum()
    correct_0 = ((targets==0)*(targets==out)).data.cpu().numpy().sum()
    total += targets.size(0)*targets.size(1) - total_0
    correct += (targets==out).data.cpu().numpy().sum() - correct_0
    
    
    
    batch.next_minibatch()
    if step%100==0:
        print('Accuracy: %1.3f' % (correct/total))
#         print ('Epoch [%d/%d], Loss: %.3f, Steps: [%d/%d], Perplexity: %5.2f' %
#        (epoch+1, num_epochs, loss.data[0], step, total_steps, np.exp(loss.data[0])))
    

Accuracy: 0.702
Accuracy: 0.697
Accuracy: 0.696
Accuracy: 0.697
Accuracy: 0.699
Accuracy: 0.700
Accuracy: 0.702
Accuracy: 0.702
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.702
Accuracy: 0.701
Accuracy: 0.701
Accuracy: 0.702
Accuracy: 0.702
Accuracy: 0.702
Accuracy: 0.702
Accuracy: 0.702
Accuracy: 0.702
Accuracy: 0.702
Accuracy: 0.702
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.702
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.703
Accuracy: 0.704
Accuracy

KeyboardInterrupt: 

In [14]:
correct/total

0.70218386100673391

In [ ]:
total = targets.size(0)*targets.size(1)
correct = (targets==out).data.cpu().numpy().sum()
print((correct-correct_0)/(total-total_0))

In [ ]:
out = outputs.view(targets.size(0),targets.size(1),-1).max(2)[1]
for i in range(len(targets[0:30])):
    line0 = vocab.idx_list_to_word_list(inputs[i].cpu().data.numpy())
    line1 = vocab.idx_list_to_word_list(targets[i].cpu().data.numpy())
    line2 = vocab.idx_list_to_word_list(out[i].cpu().data.numpy())
    for tup in zip(line0,line1,line2):
        print('\t'.join([tup[0],tup[1],tup[2],str(tup[1]==tup[2])]))
        
#     line1 = targets[i]
#     line2 = out[i]
#     print(''.join(vocab.idx_list_to_word_list(line0[:20].cpu().data.numpy())))    
#     print(''.join(vocab.idx_list_to_word_list(line1[:20].cpu().data.numpy())))
#     print(''.join(vocab.idx_list_to_word_list(line2[:20].cpu().data.numpy())))
    print('\n\n')

In [ ]:
#     with open(train_path+'list_of_files.txt') as f:
#         input_files = f.readlines()
#     i = 0
#     for file in input_files:
#         i+=1
#         with open(train_path+file.strip()) as f:
#             lines = f.readlines()
#         out_lines = []
#         for line in lines:
#             line = line.strip()
#             out_lines.append(vocab.word_list_to_idx_list(line.split())[:50])
#         batch_size = len(lines)
#         ids = torch.LongTensor(np.array(out_lines[:batch_size]))
#         states = (Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda(),
#                   Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda())

# #     for i in range(0, ids.size(1) - seq_length, seq_length):
# #         # Get batch inputs and targets
#         inputs = Variable(ids[:, :seq_length-1]).cuda()
#         targets = Variable(ids[:, 1:seq_length].contiguous()).cuda()
        
#         # Forward + Backward + Optimize
#         model.zero_grad()
#         states = detach(states)
#         outputs, states = model(inputs, states) 
#         loss = criterion(outputs, targets.view(-1))
#         loss.backward()
#         torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
#         optimizer.step()

In [ ]:
states = (Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda(),
          Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda())

In [18]:
with open('/home/irteam/users/data/150kJavaScript/data/amber-smalltalk/amber/src/Kernel-Collections.js') as f:
    txt = f.read()

In [20]:
print(txt)

define("amber_core/Kernel-Collections", ["amber/boot", "amber_core/Kernel-Objects"], function($boot){
var $core=$boot.api,nil=$boot.nil,$recv=$boot.asReceiver,$globals=$boot.globals;
$core.addPackage('Kernel-Collections');
$core.packages["Kernel-Collections"].innerEval = function (expr) { return eval(expr); };
$core.packages["Kernel-Collections"].transport = {"type":"amd","amdNamespace":"amber_core"};
$core.addClass('Association', $globals.Object, ['key', 'value'], 'Kernel-Collections');
$globals.Association.comment="I represent a pair of associated objects, a key and a value. My instances can serve as entries in a dictionary.\x0a\x0aInstances can be created with the class-side method `#key:value:`";
$core.addMethod(
$core.method({
selector: "=",
protocol: 'comparing',
fn: function (anAssociation){
var self=this;
return $core.withContext(function($ctx1) {
var $3,$2,$5,$4,$6,$1;
$3=self._class();
$ctx1.sendIdx["class"]=1;
$2=$recv($3).__eq($recv(anAssociation)._class());
$ctx1.sendIdx["

In [22]:
lexer.input(txt)

In [23]:
for token in lexer:
    print(token)

LexToken(ID,'define',1,0)
LexToken(LPAREN,'(',1,6)
LexToken(STRING,'"amber_core/Kernel-Collections"',1,7)
LexToken(COMMA,',',1,38)
LexToken(LBRACKET,'[',1,40)
LexToken(STRING,'"amber/boot"',1,41)
LexToken(COMMA,',',1,53)
LexToken(STRING,'"amber_core/Kernel-Objects"',1,55)
LexToken(RBRACKET,']',1,82)
LexToken(COMMA,',',1,83)
LexToken(FUNCTION,'function',1,85)
LexToken(LPAREN,'(',1,93)
LexToken(ID,'$boot',1,94)
LexToken(RPAREN,')',1,99)
LexToken(LBRACE,'{',1,100)
LexToken(VAR,'var',1,102)
LexToken(ID,'$core',1,106)
LexToken(EQ,'=',1,111)
LexToken(ID,'$boot',1,112)
LexToken(PERIOD,'.',1,117)
LexToken(ID,'api',1,118)
LexToken(COMMA,',',1,121)
LexToken(ID,'nil',1,122)
LexToken(EQ,'=',1,125)
LexToken(ID,'$boot',1,126)
LexToken(PERIOD,'.',1,131)
LexToken(ID,'nil',1,132)
LexToken(COMMA,',',1,135)
LexToken(ID,'$recv',1,136)
LexToken(EQ,'=',1,141)
LexToken(ID,'$boot',1,142)
LexToken(PERIOD,'.',1,147)
LexToken(ID,'asReceiver',1,148)
LexToken(COMMA,',',1,158)
LexToken(ID,'$globals',1,159)
LexToken

LexToken(ID,'$core',1,7708)
LexToken(PERIOD,'.',1,7713)
LexToken(ID,'withContext',1,7714)
LexToken(LPAREN,'(',1,7725)
LexToken(FUNCTION,'function',1,7726)
LexToken(LPAREN,'(',1,7734)
LexToken(ID,'$ctx1',1,7735)
LexToken(RPAREN,')',1,7740)
LexToken(LBRACE,'{',1,7742)
LexToken(ID,'self',1,7744)
LexToken(PERIOD,'.',1,7748)
LexToken(ID,'_subclassResponsibility',1,7749)
LexToken(LPAREN,'(',1,7772)
LexToken(RPAREN,')',1,7773)
LexToken(SEMI,';',1,7774)
LexToken(RETURN,'return',1,7776)
LexToken(ID,'self',1,7783)
LexToken(SEMI,';',1,7787)
LexToken(RBRACE,'}',1,7789)
LexToken(COMMA,',',1,7790)
LexToken(FUNCTION,'function',1,7792)
LexToken(LPAREN,'(',1,7800)
LexToken(ID,'$ctx1',1,7801)
LexToken(RPAREN,')',1,7806)
LexToken(LBRACE,'{',1,7808)
LexToken(ID,'$ctx1',1,7809)
LexToken(PERIOD,'.',1,7814)
LexToken(ID,'fill',1,7815)
LexToken(LPAREN,'(',1,7819)
LexToken(ID,'self',1,7820)
LexToken(COMMA,',',1,7824)
LexToken(STRING,'"newBucket"',1,7825)
LexToken(COMMA,',',1,7836)
LexToken(LBRACE,'{',1,7837)
Le

LexToken(ID,'each',1,15057)
LexToken(RPAREN,')',1,15061)
LexToken(LBRACE,'{',1,15062)
LexToken(RETURN,'return',1,15064)
LexToken(ID,'$core',1,15071)
LexToken(PERIOD,'.',1,15076)
LexToken(ID,'withContext',1,15077)
LexToken(LPAREN,'(',1,15088)
LexToken(FUNCTION,'function',1,15089)
LexToken(LPAREN,'(',1,15097)
LexToken(ID,'$ctx2',1,15098)
LexToken(RPAREN,')',1,15103)
LexToken(LBRACE,'{',1,15105)
LexToken(RETURN,'return',1,15107)
LexToken(ID,'$recv',1,15114)
LexToken(LPAREN,'(',1,15119)
LexToken(ID,'each',1,15120)
LexToken(RPAREN,')',1,15124)
LexToken(PERIOD,'.',1,15125)
LexToken(ID,'_asJSON',1,15126)
LexToken(LPAREN,'(',1,15133)
LexToken(RPAREN,')',1,15134)
LexToken(SEMI,';',1,15135)
LexToken(RBRACE,'}',1,15137)
LexToken(COMMA,',',1,15138)
LexToken(FUNCTION,'function',1,15140)
LexToken(LPAREN,'(',1,15148)
LexToken(ID,'$ctx2',1,15149)
LexToken(RPAREN,')',1,15154)
LexToken(LBRACE,'{',1,15156)
LexToken(ID,'$ctx2',1,15157)
LexToken(PERIOD,'.',1,15162)
LexToken(ID,'fillBlock',1,15163)
LexToken

LexToken(ID,'$1',1,22673)
LexToken(SEMI,';',1,22675)
LexToken(RBRACE,'}',1,22677)
LexToken(COMMA,',',1,22678)
LexToken(FUNCTION,'function',1,22680)
LexToken(LPAREN,'(',1,22688)
LexToken(ID,'$ctx1',1,22689)
LexToken(RPAREN,')',1,22694)
LexToken(LBRACE,'{',1,22696)
LexToken(ID,'$ctx1',1,22697)
LexToken(PERIOD,'.',1,22702)
LexToken(ID,'fill',1,22703)
LexToken(LPAREN,'(',1,22707)
LexToken(ID,'self',1,22708)
LexToken(COMMA,',',1,22712)
LexToken(STRING,'"ifEmpty:"',1,22713)
LexToken(COMMA,',',1,22723)
LexToken(LBRACE,'{',1,22724)
LexToken(ID,'aBlock',1,22725)
LexToken(COLON,':',1,22731)
LexToken(ID,'aBlock',1,22732)
LexToken(RBRACE,'}',1,22738)
LexToken(COMMA,',',1,22739)
LexToken(ID,'$globals',1,22740)
LexToken(PERIOD,'.',1,22748)
LexToken(ID,'Collection',1,22749)
LexToken(RPAREN,')',1,22759)
LexToken(RBRACE,'}',1,22760)
LexToken(RPAREN,')',1,22761)
LexToken(SEMI,';',1,22762)
LexToken(RBRACE,'}',1,22764)
LexToken(COMMA,',',1,22765)
LexToken(ID,'args',1,22767)
LexToken(COLON,':',1,22771)
Lex

LexToken(FUNCTION,'function',1,30224)
LexToken(LPAREN,'(',1,30233)
LexToken(RPAREN,')',1,30234)
LexToken(LBRACE,'{',1,30235)
LexToken(VAR,'var',1,30237)
LexToken(ID,'self',1,30241)
LexToken(EQ,'=',1,30245)
LexToken(THIS,'this',1,30246)
LexToken(SEMI,';',1,30250)
LexToken(RETURN,'return',1,30252)
LexToken(ID,'$core',1,30259)
LexToken(PERIOD,'.',1,30264)
LexToken(ID,'withContext',1,30265)
LexToken(LPAREN,'(',1,30276)
LexToken(FUNCTION,'function',1,30277)
LexToken(LPAREN,'(',1,30285)
LexToken(ID,'$ctx1',1,30286)
LexToken(RPAREN,')',1,30291)
LexToken(LBRACE,'{',1,30293)
LexToken(VAR,'var',1,30295)
LexToken(ID,'$1',1,30299)
LexToken(SEMI,';',1,30301)
LexToken(ID,'$1',1,30303)
LexToken(EQ,'=',1,30305)
LexToken(ID,'$recv',1,30306)
LexToken(LPAREN,'(',1,30311)
LexToken(ID,'self',1,30312)
LexToken(PERIOD,'.',1,30316)
LexToken(ID,'_isEmpty',1,30317)
LexToken(LPAREN,'(',1,30325)
LexToken(RPAREN,')',1,30326)
LexToken(RPAREN,')',1,30327)
LexToken(PERIOD,'.',1,30328)
LexToken(ID,'_not',1,30329)
LexT

...

LexToken(COMMA,',',1,165735)
LexToken(STRING,'"="',1,165737)
LexToken(COMMA,',',1,165740)
LexToken(STRING,'"+"',1,165742)
LexToken(COMMA,',',1,165745)
LexToken(STRING,'"copyFrom:to:"',1,165747)
LexToken(RBRACKET,']',1,165761)
LexToken(RBRACE,'}',1,165763)
LexToken(RPAREN,')',1,165764)
LexToken(COMMA,',',1,165765)
LexToken(ID,'$globals',1,165767)
LexToken(PERIOD,'.',1,165775)
LexToken(ID,'String',1,165776)
LexToken(RPAREN,')',1,165782)
LexToken(SEMI,';',1,165783)
LexToken(ID,'$core',1,165785)
LexToken(PERIOD,'.',1,165790)
LexToken(ID,'addMethod',1,165791)
LexToken(LPAREN,'(',1,165800)
LexToken(ID,'$core',1,165802)
LexToken(PERIOD,'.',1,165807)
LexToken(ID,'method',1,165808)
LexToken(LPAREN,'(',1,165814)
LexToken(LBRACE,'{',1,165815)
LexToken(ID,'selector',1,165817)
LexToken(COLON,':',1,165825)
LexToken(STRING,'"lines"',1,165827)
LexToken(COMMA,',',1,165834)
LexToken(ID,'protocol',1,165836)
LexToken(COLON,':',1,165844)
LexToken(STRING,"'split join'",1,165846)
LexToken(COMMA,',',1,165858)

LexToken(RBRACE,'}',1,177308)
LexToken(RPAREN,')',1,177309)
LexToken(COMMA,',',1,177310)
LexToken(ID,'$globals',1,177312)
LexToken(PERIOD,'.',1,177320)
LexToken(ID,'String',1,177321)
LexToken(RPAREN,')',1,177327)
LexToken(SEMI,';',1,177328)
LexToken(ID,'$core',1,177330)
LexToken(PERIOD,'.',1,177335)
LexToken(ID,'addMethod',1,177336)
LexToken(LPAREN,'(',1,177345)
LexToken(ID,'$core',1,177347)
LexToken(PERIOD,'.',1,177352)
LexToken(ID,'method',1,177353)
LexToken(LPAREN,'(',1,177359)
LexToken(LBRACE,'{',1,177360)
LexToken(ID,'selector',1,177362)
LexToken(COLON,':',1,177370)
LexToken(STRING,'"trimBoth"',1,177372)
LexToken(COMMA,',',1,177382)
LexToken(ID,'protocol',1,177384)
LexToken(COLON,':',1,177392)
LexToken(STRING,"'regular expressions'",1,177394)
LexToken(COMMA,',',1,177415)
LexToken(ID,'fn',1,177417)
LexToken(COLON,':',1,177419)
LexToken(FUNCTION,'function',1,177421)
LexToken(LPAREN,'(',1,177430)
LexToken(RPAREN,')',1,177431)
LexToken(LBRACE,'{',1,177432)
LexToken(VAR,'var',1,177434)

LexToken(RPAREN,')',1,188224)
LexToken(LBRACE,'{',1,188225)
LexToken(VAR,'var',1,188227)
LexToken(ID,'self',1,188231)
LexToken(EQ,'=',1,188235)
LexToken(THIS,'this',1,188236)
LexToken(SEMI,';',1,188240)
LexToken(RETURN,'return',1,188297)
LexToken(ID,'$core',1,188304)
LexToken(PERIOD,'.',1,188309)
LexToken(ID,'withContext',1,188310)
LexToken(LPAREN,'(',1,188321)
LexToken(FUNCTION,'function',1,188322)
LexToken(LPAREN,'(',1,188330)
LexToken(ID,'$ctx1',1,188331)
LexToken(RPAREN,')',1,188336)
LexToken(LBRACE,'{',1,188338)
LexToken(RETURN,'return',1,188363)
LexToken(ID,'String',1,188370)
LexToken(PERIOD,'.',1,188376)
LexToken(ID,'fromCharCode',1,188377)
LexToken(LPAREN,'(',1,188389)
LexToken(ID,'anInteger',1,188390)
LexToken(RPAREN,')',1,188399)
LexToken(SEMI,';',1,188400)
LexToken(RETURN,'return',1,188402)
LexToken(ID,'self',1,188409)
LexToken(SEMI,';',1,188413)
LexToken(RBRACE,'}',1,188470)
LexToken(COMMA,',',1,188471)
LexToken(FUNCTION,'function',1,188473)
LexToken(LPAREN,'(',1,188481)
Le

LexToken(RETURN,'return',1,200102)
LexToken(ID,'self',1,200109)
LexToken(SEMI,';',1,200113)
LexToken(RBRACE,'}',1,200170)
LexToken(COMMA,',',1,200171)
LexToken(FUNCTION,'function',1,200173)
LexToken(LPAREN,'(',1,200181)
LexToken(ID,'$ctx1',1,200182)
LexToken(RPAREN,')',1,200187)
LexToken(LBRACE,'{',1,200189)
LexToken(ID,'$ctx1',1,200190)
LexToken(PERIOD,'.',1,200195)
LexToken(ID,'fill',1,200196)
LexToken(LPAREN,'(',1,200200)
LexToken(ID,'self',1,200201)
LexToken(COMMA,',',1,200205)
LexToken(STRING,'"add:in:"',1,200206)
LexToken(COMMA,',',1,200215)
LexToken(LBRACE,'{',1,200216)
LexToken(ID,'anObject',1,200217)
LexToken(COLON,':',1,200225)
LexToken(ID,'anObject',1,200226)
LexToken(COMMA,',',1,200234)
LexToken(ID,'anotherObject',1,200235)
LexToken(COLON,':',1,200248)
LexToken(ID,'anotherObject',1,200249)
LexToken(RBRACE,'}',1,200262)
LexToken(COMMA,',',1,200263)
LexToken(ID,'$globals',1,200264)
LexToken(PERIOD,'.',1,200272)
LexToken(ID,'Set',1,200273)
LexToken(RPAREN,')',1,200276)
LexToke

KeyboardInterrupt: 

In [ ]:
# Sampling
sample_path = '/home/irteam/users/data/150kJavaScript/samples.txt'
with open(sample_path, 'w') as f:
    # Set intial hidden ane memory states
    state = (Variable(torch.zeros(num_layers, 1, hidden_size)).cuda(),
         Variable(torch.zeros(num_layers, 1, hidden_size)).cuda())

    # Select one word id randomly
    prob = torch.ones(vocab_size)
    input = Variable(torch.multinomial(prob, num_samples=1).unsqueeze(1),
                     volatile=True).cuda()

    for i in range(num_samples):
        # Forward propagate rnn 
        output, state = model(input, state)
        
        # Sample a word id
        prob = output.squeeze().data.exp().cpu()
        word_id = torch.multinomial(prob, 1)[0]
        
        # Feed sampled word id to next time step
        input.data.fill_(word_id)
        
        # File write
        word = corpus.dictionary.idx2word[word_id]
        word = '\n' if word == '<eos>' else word + ' '
        f.write(word)

        if (i+1) % 100 == 0:
            print('Sampled [%d/%d] words and save to %s'%(i+1, num_samples, sample_path))

In [ ]:
# Testing
for epoch in range(1):
#     states = (Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda(),
#               Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda())
    batch.next_epoch(batch_size)
    batch.initialize_states(num_layers, hidden_size)
    step=0
    while(batch.epoch_end==0):
        step+=1
        # update the minibatch inputs / outputs
        batch.get_minibatch(0)
        inputs_np = np.array([vocab.word_list_to_idx_list(line) for
                             line in batch.batch_in],dtype=int)
        targets_np = np.array([vocab.word_list_to_idx_list(line) for
                             line in batch.batch_in],dtype=int)
        inputs = Variable(torch.LongTensor(inputs_np)).cuda()
        targets = Variable(torch.LongTensor(targets_np)).cuda()
        
        outputs, states = model(inputs,batch.states)
        batch.states = detach(states)
        break

In [ ]:
out = outputs.view(batch_size,seq_length,-1).max(2)[1]

In [ ]:
input

In [ ]:
for i in range(len(targets)):
    line0 = inputs[i]
    line1 = targets[i]
    line2 = out[i]
    print(''.join(vocab.idx_list_to_word_list(line0.cpu().data.numpy())))    
    print(''.join(vocab.idx_list_to_word_list(line1.cpu().data.numpy())))
    print(''.join(vocab.idx_list_to_word_list(line2.cpu().data.numpy())))
    print('\n\n')

In [ ]:
inputs_np = ids.cpu().numpy()
targets_np = targets.cpu().data.numpy()
predicted_np = predicted_outs.cpu().data.numpy()

In [ ]:
for i in range(targets.size(0)):
    print(vocab.idx_list_to_word_list(inputs_np[i]))
    print(vocab.idx_list_to_word_list(predicted_np[i]))
    print(vocab.idx_list_to_word_list(targets_np[i]))
    print('\n')